In [ ]:
import pandas as pd
url = 'https://github.com/mattharrison/datasets/raw/master/data/vehicles.csv.zip'
df = pd.read_csv(url, engine='pyarrow', dtype_backend='pyarrow')


In [16]:
make = df.make
city_mpg = df.city08

In [3]:
make

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: string[pyarrow]

In [4]:
make.value_counts()

make
Chevrolet                           4003
Ford                                3371
Dodge                               2583
GMC                                 2494
Toyota                              2071
                                    ... 
Grumman Allied Industries              1
Environmental Rsch and Devp Corp       1
General Motors                         1
Goldacre                               1
Isis Imports Ltd                       1
Name: count, Length: 136, dtype: int64[pyarrow]

In [5]:
make.shape, make.nunique() # number of entries,number of unique entries

((41144,), 136)

In [6]:
cat_make = make.astype('category')

In [7]:
make.memory_usage(deep=True)

425767

In [8]:
cat_make.memory_usage(deep=True)

88701

In [9]:
%%timeit
cat_make.str.upper()

198 μs ± 11.8 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [10]:
%%timeit
make.str.upper()

443 μs ± 8.51 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [11]:
old_make = make.astype(str)

In [12]:
%%timeit
old_make.str.upper()

1.87 ms ± 48.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [44]:
labels = ["Very Low", "Low", "Lower-Mid", "Mid", "Upper-Mid", "High", "Higher", "Very High", "Extreme", "Ultra"]
pd.cut(city_mpg, 10, labels=labels).value_counts(sort=False)

city08
Very Low     30872
Low           9667
Lower-Mid      367
Mid             54
Upper-Mid       11
High            48
Higher          32
Very High       26
Extreme         55
Ultra           12
Name: count, dtype: int64

In [58]:
make_type = pd.CategoricalDtype( # makes the categories
    categories=sorted(make.unique()), ordered=True # sorts them, by unique
)

In [46]:
ordered_make = make.astype(make_type)
ordered_make

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: category
Categories (136, object): ['AM General' < 'ASC Incorporated' < 'Acura' < 'Alfa Romeo' ... 'Volvo' < 'Wallace Environmental' < 'Yugo' < 'smart']

In [56]:
make.sort_values()

358      AM General
369      AM General
19314    AM General
19316    AM General
20288    AM General
            ...    
32557         smart
32757         smart
32758         smart
34309         smart
34310         smart
Name: make, Length: 41144, dtype: string[pyarrow]

In [61]:
(make
 .astype('category')
 .cat.as_ordered()
 )

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: category
Categories (136, string[pyarrow]): [AM General < ASC Incorporated < Acura < Alfa Romeo ... Volvo < Wallace Environmental < Yugo < smart]

In [63]:
ordered_make.max()

'smart'

In [65]:
# cat_make.max()
# results in error, as is not ordered

In [66]:
ordered_make.sort_values()

20288    AM General
20289    AM General
369      AM General
358      AM General
19314    AM General
            ...    
31289         smart
31290         smart
29605         smart
22974         smart
26882         smart
Name: make, Length: 41144, dtype: category
Categories (136, object): ['AM General' < 'ASC Incorporated' < 'Acura' < 'Alfa Romeo' ... 'Volvo' < 'Wallace Environmental' < 'Yugo' < 'smart']

In [71]:
%%timeit
cat_make.cat.rename_categories(
    [c.lower() for c in cat_make.cat.categories]
)

78.3 μs ± 1.77 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [72]:
%%timeit
ordered_make.cat.rename_categories(
    {c: c.lower() for c in ordered_make.cat.categories}
)

41.7 μs ± 1.2 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [73]:
%%timeit
ordered_make.cat.rename_categories(
    lambda c: c.lower()
)

36.2 μs ± 454 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [74]:
ordered_make.cat.reorder_categories(
    sorted(cat_make.cat.categories, key=str.lower)
)

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: category
Categories (136, object): ['Acura' < 'Alfa Romeo' < 'AM General' < 'American Motors Corporation' ... 'Volvo' < 'VPG' < 'Wallace Environmental' < 'Yugo']

In [75]:
ordered_make.head(100).value_counts()

make
Dodge                        17
Oldsmobile                    8
Ford                          8
Buick                         7
Chevrolet                     5
                             ..
Grumman Allied Industries     0
Goldacre                      0
Geo                           0
Genesis                       0
smart                         0
Name: count, Length: 136, dtype: int64

In [79]:
(cat_make
 .head(100)
 .groupby(cat_make.head(100),observed=False)
 .first())

make
AM General                           <NA>
ASC Incorporated                     <NA>
Acura                                <NA>
Alfa Romeo                     Alfa Romeo
American Motors Corporation          <NA>
                                  ...    
Volkswagen                     Volkswagen
Volvo                               Volvo
Wallace Environmental                <NA>
Yugo                                 <NA>
smart                                <NA>
Name: make, Length: 136, dtype: category
Categories (136, string[pyarrow]): [AM General, ASC Incorporated, Acura, Alfa Romeo, ..., Volvo, Wallace Environmental, Yugo, smart]

In [80]:
(make
 .head(100)
 .groupby(make.head(100))
 .first())

make
Alfa Romeo          Alfa Romeo
Audi                      Audi
BMW                        BMW
Buick                    Buick
CX Automotive    CX Automotive
Cadillac              Cadillac
Chevrolet            Chevrolet
Chrysler              Chrysler
Dodge                    Dodge
Ferrari                Ferrari
Ford                      Ford
Hyundai                Hyundai
Infiniti              Infiniti
Lexus                    Lexus
Mazda                    Mazda
Mercury                Mercury
Nissan                  Nissan
Oldsmobile          Oldsmobile
Plymouth              Plymouth
Pontiac                Pontiac
Rolls-Royce        Rolls-Royce
Subaru                  Subaru
Toyota                  Toyota
Volkswagen          Volkswagen
Volvo                    Volvo
Name: make, dtype: string[pyarrow]

In [81]:
ordered_make.iloc[0]

'Alfa Romeo'

In [82]:
ordered_make.iloc[[0]]

0    Alfa Romeo
Name: make, dtype: category
Categories (136, object): ['AM General' < 'ASC Incorporated' < 'Acura' < 'Alfa Romeo' ... 'Volvo' < 'Wallace Environmental' < 'Yugo' < 'smart']

In [91]:
s = pd.Series(["BMW", "Ford", "BMW", "Toyota"], index=[2000, 2001, 2002, 2003])

(s.groupby(s, observed=True)
.first()
 )

BMW          BMW
Ford        Ford
Toyota    Toyota
dtype: object

In [99]:
def generalise_topn(ser, n=5, other='Other'): # three parameters
    topn = ser.value_counts().index[:n] # the top n index values sorted by value_counts()
    if isinstance(ser.dtype, pd.CategoricalDtype): # checks whether ser.dtype is a CategoricalDtype
        ser = ser.cat.set_categories( # reset the categories to be:
            topn.set_categories(list(topn)+[other]) # in a list of the topn with the specified 'other'
        )
    return ser.where(ser.isin(topn), other) # where in topn, return as is, otherwise, return specified 'other'

In [93]:
make.value_counts().index[:5]

Index(['Chevrolet', 'Ford', 'Dodge', 'GMC', 'Toyota'], dtype='string[pyarrow]', name='make')

In [95]:
make.value_counts().index

Index(['Chevrolet', 'Ford', 'Dodge', 'GMC', 'Toyota', 'BMW', 'Mercedes-Benz',
       'Nissan', 'Volkswagen', 'Mitsubishi',
       ...
       'Karma', 'Koenigsegg', 'Aurora Cars Ltd', 'RUF Automobile',
       'JBA Motorcars, Inc.', 'Grumman Allied Industries',
       'Environmental Rsch and Devp Corp', 'General Motors', 'Goldacre',
       'Isis Imports Ltd'],
      dtype='string[pyarrow]', name='make', length=136)

In [114]:
cat_make.pipe(generalise_topn, n=20, other='NA')

0            NA
1            NA
2         Dodge
3         Dodge
4        Subaru
          ...  
41139    Subaru
41140    Subaru
41141    Subaru
41142    Subaru
41143    Subaru
Name: make, Length: 41144, dtype: category
Categories (21, object): ['Chevrolet', 'Ford', 'Dodge', 'GMC', ..., 'Volvo', 'Hyundai', 'Chrysler', 'NA']

In [128]:
def generalise_topn(ser, n=5, other='Other'):
    topn = ser.value_counts().index[:n]
    if isinstance(ser.dtype, pd.CategoricalDtype):
        ser = ser.cat.set_categories(
            topn.set_categories(list(topn) + [other])
        )
    return ser.where(ser.isin(topn), other)

In [129]:
cat_make.pipe(generalise_topn, n=5, other='NA')

0           NA
1           NA
2        Dodge
3        Dodge
4           NA
         ...  
41139       NA
41140       NA
41141       NA
41142       NA
41143       NA
Name: make, Length: 41144, dtype: category
Categories (6, object): ['Chevrolet', 'Ford', 'Dodge', 'GMC', 'Toyota', 'NA']

In [130]:
def generalise_mapping(ser, mapping, default):
    seen = None
    res = ser.astype(str)
    for old, new in mapping.items():
        mask = ser.str.contains(old)
        if seen is None:
            seen = mask
        else:
            seen |= mask
        res = res.where(~mask, new)
    res = res.where(seen, default)
    return res.astype('category')

In [131]:
generalise_mapping(cat_make, {'Ford': 'US', 'Tesla': 'US',
                              'Chevrolet': 'US', 'Dodge': 'US',
                              'Oldsmobile': 'US', 'Plymouth': 'US',
                              'BMW': 'German'}, 'Other')

0        Other
1        Other
2           US
3           US
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: category
Categories (3, object): ['German', 'Other', 'US']